In [1]:
import battery_parser as bp
import os
import pandas as pd
import numpy as np
import re
import copy
import itertools

In [2]:

root_dir  = r'D:\!Science\Физтех\Циклирования\28.05.24 - результаты'



In [13]:
files = bp.list_files(directory=root_dir, filetype='xlsx') #TODO добавить ключ, по которому можно будет регулировать глубину погружения


In [80]:
patterns = [r'([A-Z]{3})', r'([^\\]*)', r'([0-9][^\\]*)', r'.?([0-9]?[0-9])', r'.*-(13[0-9])-.-([0-9]).*-([0-9]{14})']
column_names = ['Chem','Tech', 'Step', 'Pouch','Potentiostat', 'Channel', 'Datetime_id']
pattern = r'\\'.join(patterns)
print(pattern)
parser = Regex_parse()
result = parser(strings=files,pattern=pattern,column_names= column_names)
result

([A-Z]{3})\\([^\\]*)\\([0-9][^\\]*)\\.?([0-9]?[0-9])\\.*-(13[0-9])-.-([0-9]).*-([0-9]{14})
Warning! For 
 D:\!Science\Физтех\Циклирования\28.05.24 - результаты\LFP\Заявка №73 Старение+45 2-3\4. п.5 Циклирование 28 дней\разряд Приложение Б\BTS82-133-1-7-99-20240416085854-Разряд профилем Б.xlsx 
 found 0 entries!
Warning! For 
 D:\!Science\Физтех\Циклирования\28.05.24 - результаты\LFP\Заявка №73 Старение+45 2-3\4. п.5 Циклирование 28 дней\разряд Приложение Б\BTS82-133-1-8-99-20240416085912-Разряд профилем Б.xlsx 
 found 0 entries!


,Chem,Tech,Step,Pouch,Potentiostat,Channel,Datetime_id,Path
0,LFP,Заявка №73 Старение+45 2-3,1. Пункт 1-4.1,2,130,1,20240408143806,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
1,LFP,Заявка №73 Старение+45 2-3,2. 4.1,2,130,1,20240412131532,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
2,LFP,Заявка №76 Старение НКУ,4. п 5 Циклирование,8,130,1,20240417095316,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
3,LFP,Заявка №76 Старение НКУ,5. Пункт 6,8,130,1,20240514100637,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
4,LFP,Заявка №73 Старение+45 2-3,1. Пункт 1-4.1,3,130,2,20240408143819,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
...,...,...,...,...,...,...,...,...
117,LFP,Заявка №73 Старение+45 2-3,4. п.5 Циклирование 28 дней,3,133,8,20240416103846,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
118,LFP,Заявка №73 Старение+45 2-3,4. п.5 Циклирование 28 дней,3,133,8,20240416103846,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
119,LFP,Заявка №73 Старение+45 2-3,4. п.5 Циклирование 28 дней,3,133,8,20240416103846,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
120,LFP,Заявка №73 Старение+45 2-3,5. Испытание рабочих характеристик,3,133,8,20240513133513,D:\!Science\Физтех\Циклирования\28.05.24 - рез...


In [81]:
type_convert = {i:str for i in result.columns}
result = result.astype({'Pouch':int}, copy=True)
result = result.astype(type_convert, copy=True)

result['ID'] = result['Chem']+result['Pouch']
result.insert(0, 'ID', result.pop('ID'))


thermos ={'1':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
          '2':['LFP2', 'LFP3', 'NMC6', 'NMC7', 'LTO11', 'LTO12'],
          '3':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7']}
protocol = {'gost45':['NMC6', 'NMC7', 'LTO11', 'LTO12', 'LFP2', 'LFP3'],
          'gost60':['NMC12', 'NMC13', 'LTO13', 'LTO3', 'LFP6', 'LFP7'],
          'Imax45':['NMC14', 'NMC15', 'LTO7', 'LTO6', 'LFP4', 'LFP5'],
          'CCroom':['NMC10', 'NMC11', 'LTO4', 'LTO5', 'LFP1', 'LFP8']}


In [83]:
if check_dict_intersection(thermos) or check_dict_intersection(protocol):
    print('Intersection!')

class Find_key:
    def __init__(self, check_dict):
        self.dictionary = check_dict


    def __call__(self, element):
        result = [key for key, value in self.dictionary.items() if element in value]
        if len(result) == 1:
            return result[0]
        return result

thermos_creator = Find_key(thermos)
technique_creator = Find_key(protocol)

In [84]:
result['Protocol'] = result['ID'].apply(technique_creator)
result.insert(2, 'Protocol', result.pop('Protocol'))
result['Thermos'] = result['ID'].apply(thermos_creator)
result.insert(6, 'Thermos', result.pop('Thermos'))


In [85]:
result.sort_values('Step', inplace=True)
result

,ID,Chem,Protocol,Tech,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path
60,LTO3,LTO,gost60,7. Заявка № 71 К3 К13,1 п1-4.1,3,3,131,7,20240403111709,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
65,LTO13,LTO,gost60,7. Заявка № 71 К3 К13,1 п1-4.1,13,3,131,8,20240403111718,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
95,NMC7,NMC,gost45,Заявка №57 KAMAZ 006-007,1. Пункт 1-4.1,7,2,133,2,20240314161946,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
91,NMC6,NMC,gost45,Заявка №57 KAMAZ 006-007,1. Пункт 1-4.1,6,2,133,1,20240314161930,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
52,NMC15,NMC,Imax45,Заявка №64 C-NMC №014 №015,1. Пункт 1-4.1,15,1,131,6,20240325102602,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
...,...,...,...,...,...,...,...,...,...,...,...
115,LFP2,LFP,gost45,Заявка №73 Старение+45 2-3,5. Испытание рабочих характеристик,2,2,133,7,20240513133440,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
85,LFP6,LFP,gost60,Заявка №75 Старение +60,5. Испытание рабочих характеристик,6,3,132,7,20240520102038,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
110,LFP5,LFP,Imax45,Заявка №74 Старение +45 4-5,5. Испытание рабочих характеристик,5,1,133,6,20240513144931,D:\!Science\Физтех\Циклирования\28.05.24 - рез...
81,LFP1,LFP,CCroom,Заявка №76 Старение НКУ,5. Пункт 6,1,[],132,6,20240514100556,D:\!Science\Физтех\Циклирования\28.05.24 - рез...


In [86]:
#TODO Перенести потом в реализацию
test_mask = result['Step'].str.findall(r'1.*1.*-.*4\.').explode().notna()
result['Start_time'] = pd.to_datetime(result['Datetime_id'])
time_cycling = pd.Timedelta('180 days')
result['Start_time'][test_mask].sort_values()+time_cycling

91   2024-09-10 16:19:30
95   2024-09-10 16:19:46
22   2024-09-21 10:01:27
28   2024-09-21 10:01:38
34   2024-09-21 10:17:03
40   2024-09-21 10:17:17
46   2024-09-21 10:25:50
52   2024-09-21 10:26:02
23   2024-09-30 11:05:46
29   2024-09-30 11:05:58
35   2024-09-30 11:11:26
41   2024-09-30 11:11:36
47   2024-09-30 11:14:05
53   2024-09-30 11:14:15
60   2024-09-30 11:17:09
65   2024-09-30 11:17:18
0    2024-10-05 14:38:06
4    2024-10-05 14:38:19
6    2024-10-05 14:48:13
8    2024-10-05 14:48:24
10   2024-10-05 14:57:29
13   2024-10-05 14:57:42
16   2024-10-06 16:14:38
19   2024-10-06 16:14:49
Name: Start_time, dtype: datetime64[ns]

In [87]:
result

,ID,Chem,Protocol,Tech,Step,Pouch,Thermos,Potentiostat,Channel,Datetime_id,Path,Start_time
60,LTO3,LTO,gost60,7. Заявка № 71 К3 К13,1 п1-4.1,3,3,131,7,20240403111709,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-04-03 11:17:09
65,LTO13,LTO,gost60,7. Заявка № 71 К3 К13,1 п1-4.1,13,3,131,8,20240403111718,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-04-03 11:17:18
95,NMC7,NMC,gost45,Заявка №57 KAMAZ 006-007,1. Пункт 1-4.1,7,2,133,2,20240314161946,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-03-14 16:19:46
91,NMC6,NMC,gost45,Заявка №57 KAMAZ 006-007,1. Пункт 1-4.1,6,2,133,1,20240314161930,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-03-14 16:19:30
52,NMC15,NMC,Imax45,Заявка №64 C-NMC №014 №015,1. Пункт 1-4.1,15,1,131,6,20240325102602,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-03-25 10:26:02
...,...,...,...,...,...,...,...,...,...,...,...,...
115,LFP2,LFP,gost45,Заявка №73 Старение+45 2-3,5. Испытание рабочих характеристик,2,2,133,7,20240513133440,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-05-13 13:34:40
85,LFP6,LFP,gost60,Заявка №75 Старение +60,5. Испытание рабочих характеристик,6,3,132,7,20240520102038,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-05-20 10:20:38
110,LFP5,LFP,Imax45,Заявка №74 Старение +45 4-5,5. Испытание рабочих характеристик,5,1,133,6,20240513144931,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-05-13 14:49:31
81,LFP1,LFP,CCroom,Заявка №76 Старение НКУ,5. Пункт 6,1,[],132,6,20240514100556,D:\!Science\Физтех\Циклирования\28.05.24 - рез...,2024-05-14 10:05:56


In [79]:
#TODO check protocol-thermos consistence

['1. Пункт 1-4.1' '2. 4.1' '3.п 4.2' '4. п.5 Циклирование 28 дней'
 '5. Испытание рабочих характеристик']
4                          1. Пункт 1-4.1
5                                  2. 4.1
121                               3.п 4.2
117           4. п.5 Циклирование 28 дней
118           4. п.5 Циклирование 28 дней
119           4. п.5 Циклирование 28 дней
120    5. Испытание рабочих характеристик
Name: Step, dtype: object
4      1
5      2
121    2
117    3
118    3
119    3
120    4
Name: Step, dtype: int64


Testing

In [129]:
pouch = 'LFP6'
selection = result[result['ID']== 'LFP6']
print(selection)
step_id_creator(selection)

      ID Chem Protocol                     Tech  Step_id  \
10  LFP6  LFP   gost60  Заявка №75 Старение +60        1   
11  LFP6  LFP   gost60  Заявка №75 Старение +60        2   
82  LFP6  LFP   gost60  Заявка №75 Старение +60        3   
84  LFP6  LFP   gost60  Заявка №75 Старение +60        4   
83  LFP6  LFP   gost60  Заявка №75 Старение +60        4   
85  LFP6  LFP   gost60  Заявка №75 Старение +60        5   

                                               Step Pouch Thermos  \
10                                   1. Пункт 1-4.1     6       3   
11                                          2. п4.1     6       3   
82  3. Измерение динамической разрядной емкости +60     6       3   
84                      4.Циклирование 28 суток +60     6       3   
83                      4.Циклирование 28 суток +60     6       3   
85               5. Испытание рабочих характеристик     6       3   

   Potentiostat Channel     Datetime_id  \
10          130       5  20240408145729   
11       

10    1
11    2
82    2
84    3
83    3
85    4
Name: Step, dtype: int64

In [130]:
step_id = [step_id_creator(pouch) for name, pouch in result.groupby('ID')]
result['Step_id'] = pd.concat(step_id, )
result.insert(4, 'Step_id', result.pop('Step_id'))

In [136]:
for i in result.groupby(['ID', 'Step_id']):
    print(i[0], i[1].index)

('LFP1', 1) Index([19], dtype='int64')
('LFP1', 2) Index([20, 21], dtype='int64')
('LFP1', 3) Index([80], dtype='int64')
('LFP1', 4) Index([81], dtype='int64')
('LFP2', 1) Index([0], dtype='int64')
('LFP2', 2) Index([1, 116], dtype='int64')
('LFP2', 3) Index([114, 113, 112], dtype='int64')
('LFP2', 4) Index([115], dtype='int64')
('LFP3', 1) Index([4], dtype='int64')
('LFP3', 2) Index([5, 121], dtype='int64')
('LFP3', 3) Index([117, 118, 119], dtype='int64')
('LFP3', 4) Index([120], dtype='int64')
('LFP4', 1) Index([6], dtype='int64')
('LFP4', 2) Index([7, 108], dtype='int64')
('LFP4', 3) Index([106], dtype='int64')
('LFP4', 4) Index([107], dtype='int64')
('LFP5', 1) Index([8], dtype='int64')
('LFP5', 2) Index([9, 111], dtype='int64')
('LFP5', 3) Index([109], dtype='int64')
('LFP5', 4) Index([110], dtype='int64')
('LFP6', 1) Index([10], dtype='int64')
('LFP6', 2) Index([11, 82], dtype='int64')
('LFP6', 3) Index([84, 83], dtype='int64')
('LFP6', 4) Index([85], dtype='int64')
('LFP7', 1) 